### Self-Study Try-It Activity 14.1: Generating Overfit Models

By default, you scikitlearn tree models will grow until every node is pure.  To explore this, you are to build different models using the `max_depth` parameter and determine when the tree begins to overfit the data.  For depths from `max_depth = 1` until the tree is completed, keep track of the accuracy on training vs. test data and generate a plot with depths as the horizontal axis and accuracy as the vertical axis for train and test data.  

Repeat this process with different splits of the data to determine at what depth the tree begins to overfit.  Share your results with your peers and discuss your approach to generating the visualization.  What are the consequences of this overfitting for your approach to building Decision Trees?   We provide a small dataset with health data where your goal is to predict whether or not the individuals survive.

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, plot_tree

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt

In [ ]:
whickham_data = pd.read_csv('data/Whickham.txt')

In [ ]:
whickham_data.head()

The columsn `outcome` and `smoker` are categorical. Let's first convert to numerical before modellibng

In [ ]:
# Encode categorical variables
label_encoder = LabelEncoder()
whickham_data['outcome'] = label_encoder.fit_transform(whickham_data['outcome'])
whickham_data['smoker'] = label_encoder.fit_transform(whickham_data['smoker'])

# Splitting the dataset into training and testing sets
X = whickham_data[['smoker', 'age']]
y = whickham_data['outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Now let's track the accuracy of train and test at different values of `max_depth`

In [ ]:

# Define the range of max_depth to test
max_depths = range(1, 16)  # Assuming a reasonable maximum depth for illustrative purposes
train_accuracies = []
test_accuracies = []

# Train decision tree models for each depth and record their accuracies
for depth in max_depths:
    clf = DecisionTreeClassifier(max_depth=depth, random_state=42)
    clf.fit(X_train, y_train)
    y_train_pred = clf.predict(X_train)
    y_test_pred = clf.predict(X_test)
    
    # Calculate and collect accuracies
    train_accuracies.append(accuracy_score(y_train, y_train_pred))
    test_accuracies.append(accuracy_score(y_test, y_test_pred))


Now plot to visualize the training process on test and train data

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(max_depths, train_accuracies, label='Training Accuracy', marker='o')
plt.plot(max_depths, test_accuracies, label='Test Accuracy', marker='o')
plt.xlabel('Max Depth')
plt.ylabel('Accuracy')
plt.title('Accuracy vs. Tree Depth')
plt.legend()
plt.grid(True)
plt.show()

As you can see, the training data improve as we increase `max_depth`, however, test set decreases its accuracy after 5. So at 5 is the optimal `max_depth` where both test set and training set increasing.

Now let's cross validate at different splits to see if the model stable and which depth it start to overfit at different fold.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Define different test sizes to evaluate
test_sizes = [0.2, 0.4, 0.6, 0.8]

# Define the range of max_depth to test
max_depths = range(1, 16)

# Dictionary to store accuracies for different test sizes
results = {}

for test_size in test_sizes:
    # Split the data according to the current test size
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

    # Lists to store accuracies for the current split
    train_accuracies = []
    test_accuracies = []

    # Train decision tree models for each depth and record their accuracies
    for depth in max_depths:
        clf = DecisionTreeClassifier(max_depth=depth, random_state=42)
        clf.fit(X_train, y_train)
        y_train_pred = clf.predict(X_train)
        y_test_pred = clf.predict(X_test)
        
        # Calculate and collect accuracies
        train_accuracies.append(accuracy_score(y_train, y_train_pred))
        test_accuracies.append(accuracy_score(y_test, y_test_pred))
    
    # Store accuracies in the dictionary
    results[test_size] = {'train': train_accuracies, 'test': test_accuracies}

    plt.figure(figsize=(10, 6))
    plt.plot(max_depths, train_accuracies, label='Training Accuracy', marker='o')
    plt.plot(max_depths, test_accuracies, label='Test Accuracy', marker='o')
    plt.xlabel('Max Depth')
    plt.ylabel('Accuracy')
    plt.title(f'Accuracy vs. Tree Depth with {test_size}')
    plt.legend()
    plt.grid(True)
    plt.show()

